In [1]:
%load_ext autoreload
%autoreload 2

### Imports

In [2]:
import jax
print(jax.devices())
print(jax.__version__)

2023-03-07 08:38:25.130369: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-07 08:38:25.130406: W external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


[CpuDevice(id=0)]
0.4.1


In [3]:
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt

In [4]:
from jax.config import config
config.update('jax_enable_x64', True)
import keypoint_moseq as kpm

2023-03-07 08:38:55.248314: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /tigress/shruthi/miniconda3/envs/keypoint_moseq_sleap/lib/python3.8/site-packages/cv2/../../lib64:
2023-03-07 08:38:55.248349: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


### Create config files and directories for hyperparam sweep 

In [5]:
import itertools

In [16]:
hyper = {"kappa": list(np.logspace(2, 12, num=6, base=10,dtype=float)),
        "nlags": [1, 3, 5, 10],
        "alpha": [0.1, 0.5, 1, 5, 10, 50],
        "latent_dimension": list(np.arange(2, 22, 2, dtype=int))}

In [17]:
for k, v in hyper.items():
    print(v)

[100.0, 10000.0, 1000000.0, 100000000.0, 10000000000.0, 1000000000000.0]
[1, 3, 5, 10]
[0.1, 0.5, 1, 5, 10, 50]
[2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


In [7]:
base = {"kappa": 1e6,
       "alpha": 5.7, 
       "nlags": 3,
       "latent_dimension": 10}

In [8]:
from datetime import datetime

In [9]:
import os

In [10]:
base_date_string = str(datetime.now().strftime('%Y_%m_%d-%H_%M_%S'))
save_data_to = r"/scratch/gpfs/shruthi/pair_wt_gold/fitting/"
base_folder = os.path.join(save_data_to, base_date_string)
print(base_folder)

/scratch/gpfs/shruthi/pair_wt_gold/fitting/2023_03_07-08_38_55


### Setup list of sleap paths

In [11]:
video_dir = r'/scratch/gpfs/shruthi/pair_wt_gold/' 
sleap_paths = kpm.project.get_sleap_paths(video_dir)
print(sleap_paths)

/scratch/gpfs/shruthi/pair_wt_gold/190612_110639_wt_18159211_rig1.1/inference.h5
/scratch/gpfs/shruthi/pair_wt_gold/190612_110405_wt_18159111_rig2.2/inference.h5
/scratch/gpfs/shruthi/pair_wt_gold/190612_110953_wt_18159203_rig3.1/inference.h5
/scratch/gpfs/shruthi/pair_wt_gold/190612_110953_wt_18159112_rig3.2/inference.h5
/scratch/gpfs/shruthi/pair_wt_gold/190612_110405_wt_16276625_rig2.1/inference.h5
['/scratch/gpfs/shruthi/pair_wt_gold/190612_110639_wt_18159211_rig1.1/inference.h5', '/scratch/gpfs/shruthi/pair_wt_gold/190612_110405_wt_18159111_rig2.2/inference.h5', '/scratch/gpfs/shruthi/pair_wt_gold/190612_110953_wt_18159203_rig3.1/inference.h5', '/scratch/gpfs/shruthi/pair_wt_gold/190612_110953_wt_18159112_rig3.2/inference.h5', '/scratch/gpfs/shruthi/pair_wt_gold/190612_110405_wt_16276625_rig2.1/inference.h5']


In [12]:
from pathlib import Path

In [13]:
use_bodyparts = ['thorax', 'abdomen', 'wingL', 'wingR', 'forelegL4', 'forelegR4', 'midlegL4', 'midlegR4', 'hindlegL4', 'hindlegR4']

# kpm.update_config(
#     project_dir,
#     use_bodyparts=use_bodyparts,
#     anterior_bodyparts=['thorax'], posterior_bodyparts=['abdomen'],
#     latent_dimension=10, slope= -0.47, intercept= 0.236721,
#     pca_fitting_num_frames = 270000,
#     keypoint_colormap = 'cool'
# )

#### YAML is finicky about constructors to save np.float or np.int types so check types of the vals in `hyper`

In [22]:
for k, v in hyper.items():
    print(k)
    if k == 'kappa':
        print(type(v[0]))
        print(type(float(v[0])))
    print(type(v[0]))

kappa
<class 'numpy.float64'>
<class 'float'>
<class 'numpy.float64'>
nlags
<class 'int'>
alpha
<class 'float'>
latent_dimension
<class 'numpy.int64'>


#### Setup project directories with custom configs for each parameter to sweep

In [23]:
for k, v in hyper.items():
    save_folder = os.path.join(base_folder, f"sweep_{k}")
    for vi in range(len(v)):
        project_dir = os.path.join(save_folder, str(int(vi)))
        print(project_dir)
        Path(project_dir).mkdir(parents=True, exist_ok=True)
        kpm.setup_project_from_slp(project_dir,
                                   sample_slp_file=sleap_paths[0],
                                   overwrite=True) # Make sure a sample slp file exists
        if k == 'kappa':
            update_dict = {k: float(v[vi])} # YAML writing doesn't support np.float64 or np.int64 datatypes
        elif k == 'latent_dimension':
            update_dict = {k: int(v[vi])}
        print(update_dict)
        kpm.update_config(project_dir, 
                          use_bodyparts=use_bodyparts,
                          anterior_bodyparts=['thorax'],
                          posterior_bodyparts=['abdomen'],
                          latent_dimension=10,
                          slope= -0.47,
                          intercept= 0.236721,
                        pca_fitting_num_frames = 270000,
                        keypoint_colormap = 'cool',
                         **update_dict)
        print(kpm.load_config(project_dir))
    print(save_folder)
    

/scratch/gpfs/shruthi/pair_wt_gold/fitting/2023_03_07-08_38_55/sweep_kappa/0
{'kappa': 100.0}
{'bodyparts': ['head', 'thorax', 'abdomen', 'wingL', 'wingR', 'forelegL4', 'forelegR4', 'midlegL4', 'midlegR4', 'hindlegL4', 'hindlegR4', 'eyeL', 'eyeR'], 'use_bodyparts': ['thorax', 'abdomen', 'wingL', 'wingR', 'forelegL4', 'forelegR4', 'midlegL4', 'midlegR4', 'hindlegL4', 'hindlegR4'], 'skeleton': [['thorax', 'head'], ['thorax', 'abdomen'], ['thorax', 'wingL'], ['thorax', 'wingR'], ['thorax', 'forelegL4'], ['thorax', 'forelegR4'], ['thorax', 'midlegL4'], ['thorax', 'midlegR4'], ['thorax', 'hindlegL4'], ['thorax', 'hindlegR4'], ['head', 'eyeL'], ['head', 'eyeR']], 'anterior_bodyparts': ['thorax'], 'posterior_bodyparts': ['abdomen'], 'use_instance': 1, 'added_noise_level': 0.1, 'PCA_fitting_num_frames': 1000000, 'conf_threshold': 0.5, 'error_estimator': {'intercept': 0.236721, 'slope': -0.47}, 'obs_hypparams': {'nu_s': 5, 'nu_sigma': 100000.0, 'sigmasq_0': 0.1, 'sigmasq_C': 0.1}, 'ar_hypparams

{'kappa': 1000000000000.0}
{'bodyparts': ['head', 'thorax', 'abdomen', 'wingL', 'wingR', 'forelegL4', 'forelegR4', 'midlegL4', 'midlegR4', 'hindlegL4', 'hindlegR4', 'eyeL', 'eyeR'], 'use_bodyparts': ['thorax', 'abdomen', 'wingL', 'wingR', 'forelegL4', 'forelegR4', 'midlegL4', 'midlegR4', 'hindlegL4', 'hindlegR4'], 'skeleton': [['thorax', 'head'], ['thorax', 'abdomen'], ['thorax', 'wingL'], ['thorax', 'wingR'], ['thorax', 'forelegL4'], ['thorax', 'forelegR4'], ['thorax', 'midlegL4'], ['thorax', 'midlegR4'], ['thorax', 'hindlegL4'], ['thorax', 'hindlegR4'], ['head', 'eyeL'], ['head', 'eyeR']], 'anterior_bodyparts': ['thorax'], 'posterior_bodyparts': ['abdomen'], 'use_instance': 1, 'added_noise_level': 0.1, 'PCA_fitting_num_frames': 1000000, 'conf_threshold': 0.5, 'error_estimator': {'intercept': 0.236721, 'slope': -0.47}, 'obs_hypparams': {'nu_s': 5, 'nu_sigma': 100000.0, 'sigmasq_0': 0.1, 'sigmasq_C': 0.1}, 'ar_hypparams': {'K_0_scale': 10.0, 'S_0_scale': 0.01, 'nlags': 3}, 'trans_hyppa

{'kappa': 1000000000000.0}
{'bodyparts': ['head', 'thorax', 'abdomen', 'wingL', 'wingR', 'forelegL4', 'forelegR4', 'midlegL4', 'midlegR4', 'hindlegL4', 'hindlegR4', 'eyeL', 'eyeR'], 'use_bodyparts': ['thorax', 'abdomen', 'wingL', 'wingR', 'forelegL4', 'forelegR4', 'midlegL4', 'midlegR4', 'hindlegL4', 'hindlegR4'], 'skeleton': [['thorax', 'head'], ['thorax', 'abdomen'], ['thorax', 'wingL'], ['thorax', 'wingR'], ['thorax', 'forelegL4'], ['thorax', 'forelegR4'], ['thorax', 'midlegL4'], ['thorax', 'midlegR4'], ['thorax', 'hindlegL4'], ['thorax', 'hindlegR4'], ['head', 'eyeL'], ['head', 'eyeR']], 'anterior_bodyparts': ['thorax'], 'posterior_bodyparts': ['abdomen'], 'use_instance': 1, 'added_noise_level': 0.1, 'PCA_fitting_num_frames': 1000000, 'conf_threshold': 0.5, 'error_estimator': {'intercept': 0.236721, 'slope': -0.47}, 'obs_hypparams': {'nu_s': 5, 'nu_sigma': 100000.0, 'sigmasq_0': 0.1, 'sigmasq_C': 0.1}, 'ar_hypparams': {'K_0_scale': 10.0, 'S_0_scale': 0.01, 'nlags': 3}, 'trans_hyppa

TypeError: update_config() got multiple values for keyword argument 'latent_dimension'

#### Write project directories to a .txt file that can then be used to call fitting 

In [24]:
print(base_folder)

/scratch/gpfs/shruthi/pair_wt_gold/fitting/2023_03_07-08_38_55


In [26]:
txt_file_name = os.path.join(base_folder, 'array_args.txt')
with open(txt_file_name, 'w') as txt_file:
    for k, v in hyper.items():
        save_folder = os.path.join(base_folder, f"sweep_{k}")
        for vi in range(len(v)):
            project_dir = os.path.join(save_folder, str(int(vi)))
            txt_file.write(project_dir+"\n")